# LangChain & LangGraph Agent with Tool Calling and OTEL Tracing

This notebook demonstrates how to build a conversational agent using **LangChain** and **LangGraph**. The agent is equipped with several tools, including a custom retrieval tool built from web pages.

Key features highlighted:
1.  **Tool Calling**: The agent can use multiple tools, such as a calculator, a weather function, and a document retriever.
2.  **RAG (Retrieval-Augmented Generation)**: We build a retriever tool by loading, splitting, and embedding content from Lilian Weng's blog posts.
3.  **OpenTelemetry (OTEL) Tracing**: The entire process is instrumented using OpenInference, which automatically captures traces of the LangChain operations. These traces can be exported to a backend like Galileo for monitoring and debugging, or simply printed to the console.

## 1. Install Dependencies

First, we install the necessary libraries. This includes packages for OpenTelemetry, OpenInference, and the various LangChain components.

In [ ]:
!pip -q install --upgrade \
  opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation \
  openinference-instrumentation-langchain \
  langchain langchain-community langchain-openai langgraph langchain-text-splitters \
  beautifulsoup4 tiktoken

## 2. Environment Configuration

Next, we'll set up the environment variables. You'll need an OpenAI API key. 

**Tracing with Galileo (Optional)**
If you want to export traces to Galileo, you can also provide your Galileo API key, project name, and logstream. If these are not provided, tracing will still be enabled, but traces will only be printed to the console.

In [ ]:
import os
from getpass import getpass

# Set your OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")

# Optional: Galileo OTEL export settings
# Provide these via env vars to enable tracing export
# os.environ["GALILEO_API_KEY"] = getpass("Enter GALILEO_API_KEY (or leave blank to skip): ")
# os.environ["GALILEO_PROJECT"] = "your_project"
# os.environ["GALILEO_LOGSTREAM"] = "default"
# os.environ["GALILEO_OTEL_ENDPOINT"] = "https://app.galileo.ai/api/galileo/otel/traces"

galileo_api_key = os.getenv("GALILEO_API_KEY", "")
galileo_project = os.getenv("GALILEO_PROJECT", "")
galileo_logstream = os.getenv("GALILEO_LOGSTREAM", "")
otel_endpoint = os.getenv("GALILEO_OTEL_ENDPOINT", "https://app.galileo.ai/api/galileo/otel/traces")

if galileo_api_key and galileo_project and galileo_logstream:
    headers = {
        "Galileo-API-Key": galileo_api_key,
        "project": galileo_project,
        "logstream": galileo_logstream,
    }
    os.environ["OTEL_EXPORTER_OTLP_TRACES_HEADERS"] = ",".join([f"{k}={v}" for k, v in headers.items()])

print("Environment configured. Galileo tracing export is", "enabled" if os.getenv("OTEL_EXPORTER_OTLP_TRACES_HEADERS") else "disabled")

## 3. Initialize OpenTelemetry Tracing

Here, we set up the OpenTelemetry SDK. We configure a `TracerProvider` which will manage the creation of traces. 

We add two span processors:
1.  **`OTLPSpanExporter`**: This sends traces to an OTLP-compatible endpoint (like Galileo). It's only added if the necessary environment variables are set.
2.  **`ConsoleSpanExporter`**: This prints the traces directly to the console, which is useful for local debugging.

Finally, `LangChainInstrumentor().instrument()` automatically patches LangChain to create spans for all its operations.

In [ ]:
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry import trace as trace_api
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from openinference.instrumentation.langchain import LangChainInstrumentor

resource = Resource.create({"service.name": "langchain-galileo-notebook"})
tracer_provider = trace_sdk.TracerProvider(resource=resource)

# Export to Galileo if configured
if os.getenv("OTEL_EXPORTER_OTLP_TRACES_HEADERS"):
    tracer_provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint=otel_endpoint)))

# Also log spans to console for debugging
tracer_provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))

# Register the provider globally
trace_api.set_tracer_provider(tracer_provider)

# Instrument LangChain
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("LangChain OTEL tracing initialized.")

## 4. Build a Retrieval Tool

To give our agent the ability to answer questions about specific documents, we'll create a retrieval tool. This process involves a few standard RAG steps:

1.  **Load**: We use `WebBaseLoader` to fetch content from a few blog posts by Lilian Weng.
2.  **Split**: We use `RecursiveCharacterTextSplitter` to break the documents into smaller, manageable chunks.
3.  **Embed & Store**: We use `OpenAIEmbeddings` to convert the text chunks into vectors and store them in a simple `InMemoryVectorStore`.
4.  **Create Tool**: We expose the vector store as a retriever and then wrap it with `create_retriever_tool`, which makes it available for the agent to use.

In [ ]:
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

EMBED_MODEL = os.getenv("EMBED_MODEL", "text-embedding-3-small")  # or "text-embedding-3-large"
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)

urls = [
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
]

# Load web pages
docs = []
for url in urls:
    try:
        docs.extend(WebBaseLoader(url).load())
    except Exception as e:
        print(f"Failed to load {url}: {e}")

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=50)
doc_splits = text_splitter.split_documents(docs)

# Create in-memory vector store and retriever
vectorstore = InMemoryVectorStore.from_documents(doc_splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="retrieve_blog_posts",
    description="Search and return information about Lilian Weng blog posts on topics like reward hacking, hallucination, and diffusion models.",
)

print(f"Created a retriever tool with {len(doc_splits)} document splits.")

## 5. Create a ReAct Agent

Now we define the agent. We'll add two more simple tools (`multiply` and `get_weather`) using the `@tool` decorator to demonstrate how easily custom functions can be integrated.

We then use LangGraph's prebuilt `create_react_agent`. This function assembles a graph that follows the ReAct (Reasoning and Acting) logic, allowing the agent to think, use tools, observe the results, and repeat until it reaches a final answer.

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers a and b."""
    print(f"Multiplying {a} and {b}")
    return a * b

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city."""
    print(f"Getting weather for {city}")
    return f"It's always sunny in {city}!"

tools = [get_weather, multiply, retriever_tool]

agent_executor = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
)

print("ReAct agent created successfully.")

## 6. Run the Agent

Finally, let's test the agent with a few different queries. Each query is designed to trigger a different tool, demonstrating the agent's ability to reason and select the correct function for the job.

As these run, you will see the full OTEL trace data printed to the console (from our `ConsoleSpanExporter`). If you configured the Galileo exporter, the traces will also be sent there.

In [ ]:
def run_agent_message(message):
    result = agent_executor.invoke({"messages": [("user", message)]})
    final_message = result['messages'][-1]
    print(f"\n🤖 Final Answer: {final_message.content}")

print("--- Query 1: Testing the weather tool ---")
run_agent_message("what is the weather in sf")

print("\n--- Query 2: Testing the multiply tool ---")
run_agent_message("what is 12 * 666? use the tool")

print("\n--- Query 3: Testing the retriever tool ---")
run_agent_message("what does lilian weng's blog say about reward hacking?")